# 🚀 DriftWatch Complete Showcase

A comprehensive tour of everything DriftWatch v0.4.0 can do:
- All drift detectors (PSI, KS, Wasserstein, JSD, Anderson-Darling, Cramér-von Mises, Chi²)
- Feature Drift, Prediction Drift, Concept Drift
- DriftSuite unified monitoring
- Drift Explain & Visualization
- Report serialization

---

In [ ]:
import json

import numpy as np
import pandas as pd

import driftwatch

print(f"DriftWatch v{driftwatch.__version__}")

## 1. Generate Realistic Data

E-commerce customer dataset with numerical and categorical features.

In [ ]:
rng = np.random.default_rng(42)
n_ref, n_prod = 3000, 1000


def make_ecommerce_data(n, rng, session_mult=1.0, spend_mult=1.0):
    """Generate e-commerce customer features."""
    return pd.DataFrame(
        {
            "session_duration_min": (rng.lognormal(2.5, 0.8, n) * session_mult).clip(
                0.5, 120
            ),
            "pages_viewed": rng.poisson(8, n).clip(1, 50),
            "cart_value_usd": (rng.lognormal(3.5, 1.0, n) * spend_mult).clip(5, 5000),
            "days_since_last_visit": rng.exponential(14, n).clip(0, 365),
            "device_type": rng.choice(
                ["mobile", "desktop", "tablet"], n, p=[0.55, 0.35, 0.10]
            ),
        }
    )


# Reference (normal traffic)
X_ref = make_ecommerce_data(n_ref, rng)

# Production OK (same distribution)
X_prod_ok = make_ecommerce_data(n_prod, np.random.default_rng(99))

# Production DRIFT (Black Friday: longer sessions, higher spending)
X_prod_drift = make_ecommerce_data(
    n_prod,
    np.random.default_rng(77),
    session_mult=2.0,
    spend_mult=3.0,
)

print("Reference data:")
X_ref.describe().round(2)

## 2. All Drift Detectors

DriftWatch offers 7 different statistical tests:

In [ ]:
from driftwatch.detectors.registry import get_detector_by_name

detectors = [
    "psi",
    "ks",
    "wasserstein",
    "jensen_shannon",
    "anderson_darling",
    "cramer_von_mises",
]

ref_col = X_ref["session_duration_min"]
prod_col = X_prod_drift["session_duration_min"]

print(f"{'Detector':<20} {'Score':>10} {'Has Drift':>10}")
print("-" * 42)

for name in detectors:
    det = get_detector_by_name(name)
    result = det.detect(ref_col, prod_col)
    print(f"{name:<20} {result.score:>10.4f} {result.has_drift!s:>10}")

## 3. Feature Drift Monitor

The standard `Monitor` tracks changes in input feature distributions.

In [ ]:
from driftwatch import Monitor

# Select only numerical features for standard Monitor
numerical_cols = [
    "session_duration_min",
    "pages_viewed",
    "cart_value_usd",
    "days_since_last_visit",
]

monitor = Monitor(
    reference_data=X_ref[numerical_cols],
    features=numerical_cols,
)

# Normal traffic
report_ok = monitor.check(X_prod_ok[numerical_cols])
print(
    f"Normal traffic — Drift: {report_ok.has_drift()}, Ratio: {report_ok.drift_ratio():.0%}"
)

# Black Friday
report_bf = monitor.check(X_prod_drift[numerical_cols])
print(
    f"Black Friday  — Drift: {report_bf.has_drift()}, Ratio: {report_bf.drift_ratio():.0%}"
)
print(f"Drifted features: {report_bf.drifted_features()}")

print("\nDetailed report:")
print(report_bf.summary())

## 4. Prediction Drift

Track how model predictions shift — no labels needed!

In [ ]:
from driftwatch import PredictionMonitor

# Simulate purchase predictions
rng2 = np.random.default_rng(42)
y_ref_pred = rng2.random(n_ref)  # Purchase probabilities
y_prod_pred = rng2.random(n_prod) * 0.5  # Shifted down

pred_monitor = PredictionMonitor(
    reference_predictions=y_ref_pred,
    detector="ks",
)

pred_report = pred_monitor.check(y_prod_pred)

print(f"Prediction drift: {pred_report.has_drift()}")
for r in pred_report.feature_results:
    print(f"  Type: {r.drift_type.value}, Method: {r.method}, Score: {r.score:.4f}")

## 5. Concept Drift

Detect when model performance degrades.

In [ ]:
from driftwatch import ConceptMonitor

# Simulate labels and predictions
rng3 = np.random.default_rng(42)
y_ref_true = rng3.integers(0, 2, n_ref)
y_ref_pred_cls = y_ref_true.copy()
y_ref_pred_cls[:50] = 1 - y_ref_pred_cls[:50]  # 98% accuracy

y_prod_true = rng3.integers(0, 2, n_prod)
y_prod_pred_cls = rng3.integers(0, 2, n_prod)  # Random predictions (~50%)

concept_monitor = ConceptMonitor(
    task="classification",
    metrics=["accuracy", "f1", "precision", "recall"],
)

concept_report = concept_monitor.check(
    y_ref_true,
    y_ref_pred_cls,
    y_prod_true,
    y_prod_pred_cls,
)

print(f"Concept drift: {concept_report.has_drift()}\n")

print(f"{'Metric':>12} {'Ref':>8} {'Prod':>8} {'Change':>8} {'Status':>12}")
print("-" * 52)
for d in concept_monitor.performance_details:
    status = "⚠ DEGRADED" if d.has_degradation else "✓ OK"
    print(
        f"{d.metric_name:>12} {d.reference_value:>8.3f} "
        f"{d.production_value:>8.3f} {d.absolute_change:>+8.3f} {status:>12}"
    )

## 6. DriftSuite — All Three in One

The recommended way to monitor in production.

In [ ]:
from driftwatch import DriftSuite

suite = DriftSuite(
    reference_data=X_ref[numerical_cols],
    reference_predictions=y_ref_pred_cls,
    task="classification",
    performance_metrics=["accuracy", "f1"],
    model_version="ecommerce-v3.1",
)

comprehensive = suite.check(
    production_data=X_prod_drift[numerical_cols],
    production_predictions=y_prod_pred_cls,
    y_true_ref=y_ref_true,
    y_pred_ref=y_ref_pred_cls,
    y_true_prod=y_prod_true,
    y_pred_prod=y_prod_pred_cls,
)

print(comprehensive.summary())

In [ ]:
# JSON export
output = json.loads(comprehensive.to_json())
print(json.dumps(output, indent=2, default=str)[:800])

## 7. Drift Explain

Understand *why* drift happened with statistical details.

In [ ]:
from driftwatch.explain import DriftExplainer

explainer = DriftExplainer(
    reference_data=X_ref[numerical_cols],
    production_data=X_prod_drift[numerical_cols],
    report=report_bf,
)

explanation = explainer.explain()
print(explanation.summary())

---

## Summary

DriftWatch v0.4.0 provides:

| Component | Purpose |
|-----------|--------|
| `Monitor` | Feature drift detection (P(X)) |
| `PredictionMonitor` | Prediction drift detection (P(Ŷ)) |
| `ConceptMonitor` | Concept drift detection (P(Y|X)) |
| `DriftSuite` | Unified multi-drift monitoring |
| `DriftExplainer` | Statistical drift explanation |
| 7 Detectors | PSI, KS, Wasserstein, JSD, A-D, CvM, Chi² |
| 3 Integrations | FastAPI, Slack, Email, MLflow |

🔗 [Documentation](https://vincentcotella.github.io/DriftWatch/) | [GitHub](https://github.com/VincentCotella/DriftWatch)